In [10]:
#@title celeba
!python3 test_iddgan_celeba.py --dataset celeba_256 --image_size 256 --exp vq-f4-256 --epoch_id 475 --num_channels 3 \
--num_channels_dae 128 --nz 100 --z_emb_dim 256  --ch_mult 1 2 2 2  --num_timesteps 2 --num_res_blocks 2 \
--current_resolution 64 --attn_resolutions 16 --n_mlp 4 \
--AutoEncoder_config ./autoencoder/config/vq-f4.yaml --AutoEncoder_ckpt ./autoencoder/weight/vq-f4.ckpt \
--scale_factor 6.0 \
--batch_size 25 \
--compute_fid --real_img_dir pytorch_fid/celebahq_stat.npy 

64 [1, 2, 2, 2] [16]
GEN: <class 'score_sde.models.ncsnpp_generator_adagn.NCSNpp'>
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 3, 64, 64) = 12288 dimensions.
making attention of type 'vanilla' with 512 in_channels
/home/users/std/2021/21k0005/anaconda3/envs/iddgan/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/users/std/2021/21k0005/anaconda3/envs/iddgan/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
loaded pretrained LPIPS loss from taming/modules/

In [ ]:
dataset: celeba_256, exp: vq-f4-256
epoch: 450, FID: 6.047161905975514, H100?Inference time(25 batch): 284.59+/-7.73ms, Inference time(1 batch): 38.62+/-9.04ms
epoch: 475, FID: 6.298880848146553
epoch: 500, FID: 5.837853879242658, A100 Inference time(100 batch): 1410.00+/-3.73ms, Inference time(25 batch): 374.53+/-2.48ms, Inference time(batch 1): 58.38+/-9.02ms

In [2]:
#@title measure time
!python3 test_iddgan.py --dataset cifar10 --exp kl-f2-4 --epoch_id 1700 --num_channels 4 \
--num_channels_dae 128 --num_timesteps 4 --num_res_blocks 2 --nz 50 --z_emb_dim 256 \
--n_mlp 4 --ch_mult 1 2 2 --image_size 32 --current_resolution 16 --attn_resolutions 32 \
--scale_factor 105.0 --AutoEncoder_config autoencoder/config/kl-f2.yaml --AutoEncoder_ckpt autoencoder/weight/kl-f2.ckpt \
--batch_size 100 --measure_time --real_img_dir pytorch_fid/cifar10_train_stat.npy 

16 [1, 2, 2] [32]
GEN: <class 'score_sde.models.ncsnpp_generator_adagn.NCSNpp'>
making attention of type 'vanilla' with 256 in_channels
making attention of type 'vanilla' with 256 in_channels
making attention of type 'vanilla' with 256 in_channels
Working with z of shape (1, 4, 16, 16) = 1024 dimensions.
making attention of type 'vanilla' with 256 in_channels
making attention of type 'vanilla' with 256 in_channels
making attention of type 'vanilla' with 256 in_channels
making attention of type 'vanilla' with 256 in_channels
/home/users/std/2021/21k0005/anaconda3/envs/iddgan/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/users/std/2021/21k0005/anaconda3/envs/iddgan/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.1

In [ ]:
dataset: cifar10, exp: kl-f2-4
epoch: 1000, FID: 7.7001933186085125
epoch: 1100, FID: 8.0272711865656
epoch: 1200, FID: 6.965406890685074

In [1]:
#@title cumpute FID
!python3 test_iddgan.py --dataset cifar10 --exp kl-f2-4 --epoch_id 1300 --num_channels 4 \
--num_channels_dae 128 --num_timesteps 4 --num_res_blocks 2 --nz 50 --z_emb_dim 256 \
--n_mlp 4 --ch_mult 1 2 2 --image_size 32 --current_resolution 16 --attn_resolutions 32 \
--scale_factor 105.0 --AutoEncoder_config autoencoder/config/kl-f2.yaml --AutoEncoder_ckpt autoencoder/weight/kl-f2.ckpt \
--batch_size 250 --fid_only --real_img_dir pytorch_fid/cifar10_train_stat.npy 

^C
Traceback (most recent call last):
  File "/scratch/users/std/2021/21k0005/improved-ddgan/test_iddgan.py", line 12, in <module>
    from score_sde.models.ncsnpp_generator_adagn import NCSNpp, WaveletNCSNpp
  File "/scratch/users/std/2021/21k0005/improved-ddgan/score_sde/models/ncsnpp_generator_adagn.py", line 42, in <module>
    from . import dense_layer, layers, layerspp, utils
  File "/scratch/users/std/2021/21k0005/improved-ddgan/score_sde/models/layerspp.py", line 37, in <module>
    from . import dense_layer, layers, up_or_down_sampling
  File "/scratch/users/std/2021/21k0005/improved-ddgan/score_sde/models/up_or_down_sampling.py", line 15, in <module>
    from score_sde.op import upfirdn2d
  File "/scratch/users/std/2021/21k0005/improved-ddgan/score_sde/op/__init__.py", line 1, in <module>
    from .fused_act import FusedLeakyReLU, fused_leaky_relu
  File "/scratch/users/std/2021/21k0005/improved-ddgan/score_sde/op/fused_act.py", line 19, in <module>
    fused = load(
  File "

In [ ]:
#@title cumpute FID
!python3 test_iddgan.py --dataset cifar10 --exp kl-f2-4 --epoch_id 1400 --num_channels 4 \
--num_channels_dae 128 --num_timesteps 4 --num_res_blocks 2 --nz 50 --z_emb_dim 256 \
--n_mlp 4 --ch_mult 1 2 2 --image_size 32 --current_resolution 16 --attn_resolutions 32 \
--scale_factor 105.0 --AutoEncoder_config autoencoder/config/kl-f2.yaml --AutoEncoder_ckpt autoencoder/weight/kl-f2.ckpt \
--batch_size 250 --compute_fid --real_img_dir pytorch_fid/cifar10_train_stat.npy 

In [ ]:
#@title cumpute FID
!python3 test_iddgan.py --dataset cifar10 --exp kl-f2-4 --epoch_id 1500 --num_channels 4 \
--num_channels_dae 128 --num_timesteps 4 --num_res_blocks 2 --nz 50 --z_emb_dim 256 \
--n_mlp 4 --ch_mult 1 2 2 --image_size 32 --current_resolution 16 --attn_resolutions 32 \
--scale_factor 105.0 --AutoEncoder_config autoencoder/config/kl-f2.yaml --AutoEncoder_ckpt autoencoder/weight/kl-f2.ckpt \
--batch_size 250 --compute_fid --real_img_dir pytorch_fid/cifar10_train_stat.npy 

In [ ]:
#@title cumpute FID
!python3 test_iddgan.py --dataset cifar10 --exp kl-f2-4 --epoch_id 1600 --num_channels 4 \
--num_channels_dae 128 --num_timesteps 4 --num_res_blocks 2 --nz 50 --z_emb_dim 256 \
--n_mlp 4 --ch_mult 1 2 2 --image_size 32 --current_resolution 16 --attn_resolutions 32 \
--scale_factor 105.0 --AutoEncoder_config autoencoder/config/kl-f2.yaml --AutoEncoder_ckpt autoencoder/weight/kl-f2.ckpt \
--batch_size 250 --compute_fid --real_img_dir pytorch_fid/cifar10_train_stat.npy 

In [ ]:
#@title cumpute FID
!python3 test_iddgan.py --dataset cifar10 --exp kl-f2-4 --epoch_id 1700 --num_channels 4 \
--num_channels_dae 128 --num_timesteps 4 --num_res_blocks 2 --nz 50 --z_emb_dim 256 \
--n_mlp 4 --ch_mult 1 2 2 --image_size 32 --current_resolution 16 --attn_resolutions 32 \
--scale_factor 105.0 --AutoEncoder_config autoencoder/config/kl-f2.yaml --AutoEncoder_ckpt autoencoder/weight/kl-f2.ckpt \
--batch_size 250 --compute_fid --real_img_dir pytorch_fid/cifar10_train_stat.npy 

In [ ]:
#@title cumpute FID
!python3 test_iddgan.py --dataset cifar10 --exp kl-f2-4 --epoch_id 1800 --num_channels 4 \
--num_channels_dae 128 --num_timesteps 4 --num_res_blocks 2 --nz 50 --z_emb_dim 256 \
--n_mlp 4 --ch_mult 1 2 2 --image_size 32 --current_resolution 16 --attn_resolutions 32 \
--scale_factor 105.0 --AutoEncoder_config autoencoder/config/kl-f2.yaml --AutoEncoder_ckpt autoencoder/weight/kl-f2.ckpt \
--batch_size 250 --compute_fid --real_img_dir pytorch_fid/cifar10_train_stat.npy 

In [ ]:
#@title cumpute FID
!python3 test_iddgan.py --dataset cifar10 --exp kl-f2-4 --epoch_id 1900 --num_channels 4 \
--num_channels_dae 128 --num_timesteps 4 --num_res_blocks 2 --nz 50 --z_emb_dim 256 \
--n_mlp 4 --ch_mult 1 2 2 --image_size 32 --current_resolution 16 --attn_resolutions 32 \
--scale_factor 105.0 --AutoEncoder_config autoencoder/config/kl-f2.yaml --AutoEncoder_ckpt autoencoder/weight/kl-f2.ckpt \
--batch_size 250 --compute_fid --real_img_dir pytorch_fid/cifar10_train_stat.npy 

In [ ]:
#@title cumpute FID
!python3 test_iddgan.py --dataset cifar10 --exp kl-f2-4 --epoch_id 2000 --num_channels 4 \
--num_channels_dae 128 --num_timesteps 4 --num_res_blocks 2 --nz 50 --z_emb_dim 256 \
--n_mlp 4 --ch_mult 1 2 2 --image_size 32 --current_resolution 16 --attn_resolutions 32 \
--scale_factor 105.0 --AutoEncoder_config autoencoder/config/kl-f2.yaml --AutoEncoder_ckpt autoencoder/weight/kl-f2.ckpt \
--batch_size 250 --compute_fid --real_img_dir pytorch_fid/cifar10_train_stat.npy 

In [1]:
#@title FID only
!python3 test_iddgan.py --dataset cifar10 --exp kl-f2-3 --epoch_id 2000 --num_channels 4 \
--num_channels_dae 128 --num_timesteps 4 --num_res_blocks 2 --nz 50 --z_emb_dim 256 \
--n_mlp 4 --ch_mult 1 2 2 --image_size 32 --current_resolution 16 --attn_resolutions 32 \
--scale_factor 105.0 --AutoEncoder_config autoencoder/config/kl-f2.yaml --AutoEncoder_ckpt autoencoder/weight/kl-f2.ckpt \
--batch_size 250 --fid_only --real_img_dir pytorch_fid/cifar10_train_stat.npy 

16 [1, 2, 2] [32]
GEN: <class 'score_sde.models.ncsnpp_generator_adagn.NCSNpp'>
making attention of type 'vanilla' with 256 in_channels
making attention of type 'vanilla' with 256 in_channels
making attention of type 'vanilla' with 256 in_channels
Working with z of shape (1, 4, 16, 16) = 1024 dimensions.
making attention of type 'vanilla' with 256 in_channels
making attention of type 'vanilla' with 256 in_channels
making attention of type 'vanilla' with 256 in_channels
making attention of type 'vanilla' with 256 in_channels
/scratch/users/std/2021/21k0005/improved-ddgan/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/scratch/users/std/2021/21k0005/improved-ddgan/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since

In [6]:
!python3 test_iddgan_celeba.py --dataset afhq_cat --exp vq-f4-256 --epoch_id 400 --num_channels 3 \
--num_channels_dae 128 --num_timesteps 4 --num_res_blocks 2 --nz 100 --z_emb_dim 256 \
--n_mlp 4 --ch_mult 1 2 2 2 --image_size 256 --current_resolution 64 --attn_resolutions 16 \
--scale_factor 6.0 --AutoEncoder_config autoencoder/config/vq-f4.yaml --AutoEncoder_ckpt autoencoder/weight/vq-f4.ckpt \
--batch_size 50 --measure_time --compute_fid --real_img_dir data/afhq/train/cat

64 [1, 2, 2, 2] [16]
GEN: <class 'score_sde.models.ncsnpp_generator_adagn.NCSNpp'>
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 3, 64, 64) = 12288 dimensions.
making attention of type 'vanilla' with 512 in_channels
/scratch/users/std/2021/21k0005/improved-ddgan/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/scratch/users/std/2021/21k0005/improved-ddgan/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
loaded pretrained LPIPS loss from taming/modu

In [ ]:
dataset: CIFAR10, exp: kl-f2, epoch:475, FID: 9.471584868250147

In [ ]:
dataset: cifar10, exp: kl-f2-2, epoch: 1000, FID: 5.449565416646351
dataset: cifar10, exp: kl-f2-2, epoch: 1100, FID: 5.218899761556486
dataset: cifar10, exp: kl-f2-2, epoch: 1200, FID: 4.695496785675118
dataset: cifar10, exp: kl-f2-2, epoch: 1300, FID: 4.564839950266332
dataset: cifar10, exp: kl-f2-2, epoch: 1400, FID: 4.5482688011948085
dataset: cifar10, exp: kl-f2-2, epoch: 1500, FID: 4.59995502717527
dataset: cifar10, exp: kl-f2-2, epoch: 1600, FID: 4.540297830066777
dataset: cifar10, exp: kl-f2-2, epoch: 1650, FID: 4.57000753925297
dataset: cifar10, exp: kl-f2-2, epoch: 1675, FID: 4.939058043092189
dataset: cifar10, exp: kl-f2-2, epoch: 1700, FID: 4.3555737228003295, Inference time(batch 100): 301.88+/-70.94ms, Inference time(batch 1): 240.47+/-64.81ms, Inference time(batch 1000): 1179.33+/-54.45ms
dataset: cifar10, exp: kl-f2-2, epoch: 1725, FID: 4.648243839433746
dataset: cifar10, exp: kl-f2-2, epoch: 1750, FID: 5.364749508773798
dataset: cifar10, exp: kl-f2-2, epoch: 1800, FID: 6.04712383350892
dataset: cifar10, exp: kl-f2-2, epoch: 2000, FID: 8.840900641825328

In [ ]:
dataset: cifar10, exp: kl-f2-3, 
epoch: 1000, FID: 5.105137949359914
epoch: 1100, FID: 5.0770264059576675
epoch: 1200, FID: 4.96211908980996
epoch: 1300, FID: 4.437076291003905
epoch: 1400, FID: 5.160231192035496
epoch: 1500, FID: 4.176725399173108
epoch: 1525, FID: 4.17484843314071
epoch: 1550, FID: 4.144169482670179
epoch: 1575, FID: 4.0306193305124225
epoch: 1600, FID: 4.030334598838294
epoch: 1625, FID: 4.257021107989885
epoch: 1650, FID: 4.2181519657891045
epoch: 1700, FID: 4.550198534654214
epoch: 1750, FID: 4.325918065430983
epoch: 1775, FID: 4.073972311063983
epoch: 1800, FID: 4.0148707295130635
epoch: 1825, FID: 3.91055155030233 Inference time(100 batch): 59.71+/-6.66ms, Inference time(1 batch): 36.84+/-5.13ms
epoch: 1850, FID: 3.9791375357232255
epoch: 1900, FID: 4.031493533134892
epoch: 2000, FID: 4.146907299500981

In [ ]:
AFHQ-CAT vq-f4-256 epoch:400 FID = 22.459739544399042
AFHQ-CAT vq-f4-256 epoch:500 FID = 21.64908864472619

/scratch/users/std/2021/21k0005/improved-ddgan
